In [1]:
import numpy as np
import math
import cvxpy as cp
import pandas as pd

In [2]:
#initialization
A = pd.read_csv("A_3.csv")
pi = pd.read_csv("pi_dependent.csv")
A = np.array(A)
pi  = np.array(pi)
pi = pi.T
print(pi.shape)
n = 10000
m = 10
b_i = 1000
k = 50
b = np.array([1000 for _ in range(m)])  

x = cp.Variable(k)
s = cp.Variable(m)


(1, 10000)


Q4

In [3]:
pi1 = pi[:,:k]
A1 = A[:m,:k]
print(A[:,k])
print(A1.shape)
print(pi1.shape)
w = 1
def u(s):
    u_s = 0
    for i in range(m):
        u_s += w/m * cp.log(s[i])
    return u_s
    
obj = cp.Maximize(pi1@x + u(s))
const = [A1@x + s <= k/n * b_i, x>=0, x<=1]
prob = cp.Problem(obj, const)

prob.solve()
print(x.value)
print(prob.constraints[0].dual_value)
print("\nThe optimal value is", prob.value)
prob.solve(solver="SCS")
x.value

print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution corresponding to the inequality constraints is")
print(prob.constraints[0].dual_value)
print(prob.constraints[2].dual_value)



[0. 0. 1. 0. 1. 1. 0. 1. 1. 1.]
(10, 50)
(1, 50)
[-7.19666198e-09 -7.02438281e-09 -7.73367471e-09  3.61848124e-08
 -4.32615971e-09 -6.23087483e-09 -6.51137131e-09 -7.08545525e-09
  4.49427100e-01 -5.32037921e-09 -7.70612118e-09  7.44924178e-10
 -5.56628848e-09 -7.49554649e-09  4.22715503e-11 -5.88068791e-09
 -7.29229656e-09 -6.26022450e-09  9.97235553e-01 -4.58936373e-09
 -4.68311466e-09  1.35253422e-07  1.40163397e-08 -6.19236024e-09
  1.15969323e-10 -6.63431985e-09 -5.77915346e-09 -4.59751273e-09
 -6.08790519e-09  4.80794590e-01 -5.87135741e-09  9.99999983e-01
  2.52584181e-03 -5.10909461e-09  9.99999993e-01  5.13617503e-01
  9.99999992e-01 -4.74347634e-09 -5.16118467e-11  4.83648907e-01
  9.99999968e-01 -6.72152294e-09  9.99999993e-01  9.97769655e-01
  9.99999876e-01  1.27965796e-09  3.37289430e-02  9.99999944e-01
  9.99999996e-01  2.47540047e-08]
[34.11751686 44.83465361 18.89477484  2.81241286 48.39357632  0.20589859
  0.09972473 24.64264651 36.57907666 16.69175834]

The optimal v

Q8

In [4]:
l_decision = []
b_k = b
d = b/n
price = [0 for _ in range(m)]
x = 0
flag = 1
AT = A.T

# k = 0 initialization
l_decision.append(0)
for k in range(1,n):
    alpha = k**(-0.5)
    # update price first
    price = price + alpha*(AT[k]*x-d)
    for i in range(price.shape[0]):
        if price[i] < 0:
            price[i] = 0
    flag = 1
    # Check whether remaining certain resources needed for k
    for i in range(len(b_k)):
        if A[:,k][i] > b_k[i]:
            # print("cannot satisfy")
            l_decision.append(0)
            flag = 0
            break
    if flag == 0:
        continue

    # Decide whether to accept (k)th request and the remaining resource
    if pi[0][k] > AT[k].dot(price):
        x = 1
        b_k = b_k - A[:,k]
    else:
        x = 0
    l_decision.append(x)

    # Update the dual price after making the decision
    A1 = A[:, :k+1]
    pik = pi[:,:k+1]
    # print("ak", A[:,k+1])
    # print("bk", b_k) 
    Anow = A1.T
    # y = cp.Variable((m+k+1))
    e = np.ones(k+1)
    bnow = b_k * (float((k+1)/(n-k+1)))
    pNow = np.reshape(pik, (k+1,)) 

l_decision
print(len(l_decision))

10000


In [5]:
x = cp.Variable(n)
ee = np.ones(n)
objective = cp.Maximize(pi@x)
constraints = [A@x <= b, x >= 0, x <= ee]
prob = cp.Problem(objective, constraints)

prob.solve()
x.value
opv1 = prob.value
print(opv1)

42255092.735269636


In [6]:
print(len(l_decision))
print(pi.shape)
# print(pi@l_decision)
# print(pi@l_decision - opv1)

10000
(1, 10000)


In [7]:

opv2 = (pi.dot(l_decision))[0]
print("optimal value:", opv2)
print("competetive ratio", opv2/opv1)

optimal value: 4498324.485462521
competetive ratio 0.10645638653889034
